In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import r2_score

In [27]:
class myRidge:

    def __init__(self,alpha=0.1):
        self.alpha_ = alpha
        self.beta_entire_ = None
        self.beta_ = None
        self.intercept_ = None

    def fit(self,x_train,y_train):
        x_train = x_train.to_numpy()
        ones = np.ones((x_train.shape[0],1))
        x_train = np.hstack((ones,x_train))
        y_train = y_train.to_numpy().reshape(-1,1)

        a = np.dot(x_train.T,x_train) + (self.alpha_*np.eye(x_train.shape[1]))
        ainv = np.linalg.inv(a)
        b = np.dot(x_train.T,y_train)

        self.beta_entire_ = np.dot(ainv,b)
        self.intercept_ = self.beta_entire_[0]
        self.beta_ = self.beta_entire_[1:]

        print(self.intercept_ , self.beta_)

    def predict(self,x_test):
        x_test = x_test.to_numpy()
        ones = np.ones((x_test.shape[0],1))
        x_test = np.hstack((ones,x_test))
        
        yhat = np.dot(x_test,self.beta_entire_)
        return yhat

In [3]:
x,y = load_diabetes(return_X_y=True,as_frame=True)

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [6]:
rr = Ridge(alpha=0.1)

In [7]:
rr.fit(x_train,y_train)

Ridge(alpha=0.1)

In [8]:
y_rr_pred = rr.predict(x_test)

In [9]:
r2_score(y_test,y_rr_pred)

0.4294232200423981

In [10]:
rr.coef_

array([  31.62862333, -224.04289996,  496.35598161,  304.97137183,
        -51.18604713,  -60.47562534, -193.6731529 ,   98.1648489 ,
        391.70049577,  130.87014988])

In [28]:
myrr = myRidge()

In [29]:
myrr.fit(x_train,y_train)

[150.91733428] [[  31.624523  ]
 [-224.05392495]
 [ 496.35096737]
 [ 304.97515265]
 [ -51.1896705 ]
 [ -60.48414221]
 [-193.65837306]
 [  98.2020873 ]
 [ 391.69239672]
 [ 130.85498484]]


In [40]:
y_pred_myrr = myrr.predict(x_test)

In [41]:
r2_score(y_test,y_pred_myrr)

0.42933964535839453

In [77]:
class myRidgeGD:
    
    def __init__(self,alpha=0.001,epochs=500,l_rate=0.005):
        self.alpha_ = alpha
        self.epochs_ = epochs
        self.l_rate_ = l_rate
        self.beta_ = None
        self.coef_ = None
        self.intercept_ = None

    def fit(self,x_train,y_train):
        x_train = x_train.to_numpy()
        y_train = y_train.to_numpy().reshape(-1,1)
        x_train = np.insert(x_train,0,1,axis=1)
        beta = np.ones(x_train.shape[1]).reshape(-1,1)
        beta[0] = 0

        for i in range(self.epochs_):
            beta_reg = beta.copy()
            beta_reg[0] = 0
            gradient = x_train.T @ (x_train @ beta - y_train) + (self.alpha_ * beta_reg)
            beta = beta - (self.l_rate_*gradient)
            
        self.beta_ = beta
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]

    def predict(self,x_test):
        x_test = x_test.to_numpy()
        x_test = np.insert(x_test,0,1,axis=1)
        yhat = np.dot(x_test,self.beta_)
        return yhat

In [78]:
myrrgd = myRidgeGD()

In [79]:
myrrgd.fit(x_train,y_train)

In [80]:
y_pred_rrgd = myrrgd.predict(x_test)

In [81]:
r2_score(y_test,y_pred_rrgd)

0.4235269879135163

In [83]:
class myRidgeSGD:
    
    def __init__(self,alpha=0.001,epochs=25,l_rate=0.005):
        self.alpha_ = alpha
        self.epochs_ = epochs
        self.l_rate_ = l_rate
        self.beta_ = None
        self.coef_ = None
        self.intercept_ = None

    def fit(self,x_train,y_train):
        x_train = x_train.to_numpy()
        y_train = y_train.to_numpy().reshape(-1,1)
        x_train = np.insert(x_train,0,1,axis=1)
        beta = np.ones(x_train.shape[1]).reshape(-1,1)
        beta[0] = 0
        rows = x_train.shape[0]
        for i in range(self.epochs_):
            for j in range(rows):
                idx = np.random.randint(0,rows)
                beta_reg = beta.copy()
                beta_reg[0] = 0
                
                xi = x_train[idx].reshape(1,-1)
                yi = y_train[idx].reshape(1,-1)
                gradient = xi @ (xi @ beta - yi) + (self.alpha_ * beta_reg)
                beta = beta - (self.l_rate_*gradient)
            
        self.beta_ = beta
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]

    def predict(self,x_test):
        x_test = x_test.to_numpy()
        x_test = np.insert(x_test,0,1,axis=1)
        yhat = np.dot(x_test,self.beta_)
        return yhat

In [ ]:
class myRidgeMBGD:
    
    def __init__(self,alpha=0.001,epochs=25,l_rate=0.005,batch_size = 25):
        self.batch_size_ = batch_size
        self.alpha_ = alpha
        self.epochs_ = epochs
        self.l_rate_ = l_rate
        self.beta_ = None
        self.coef_ = None
        self.intercept_ = None

    def fit(self,x_train,y_train):
        x_train = x_train.to_numpy()
        y_train = y_train.to_numpy().reshape(-1,1)
        x_train = np.insert(x_train,0,1,axis=1)
        beta = np.ones(x_train.shape[1]).reshape(-1,1)
        beta[0] = 0
        rows = x_train.shape[0]
        iterations = int(rows/self.batch_size_)
        for i in range(self.epochs_):
            start = 0
            for j in range(iterations):
                beta_reg = beta.copy()
                beta_reg[0] = 0

                xi = x_train[start : start + self.batch_size_ , :]
                yi = y_train[start : start + self.batch_size_ ]
                start += self.batch_size_
                gradient = xi.T @ (xi @ beta - yi) + (self.alpha_ * beta_reg)
                beta = beta - (self.l_rate_*gradient)
            
        self.beta_ = beta
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]

    def predict(self,x_test):
        x_test = x_test.to_numpy()
        x_test = np.insert(x_test,0,1,axis=1)
        yhat = np.dot(x_test,self.beta_)
        return yhat